In [22]:
import pandas as pd
from sqlalchemy import text
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

In [23]:
server_name   = "localhost"
database_name = "bookstore"

# Convert to string acceptable by create_engine
connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes"
url_string = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(url_string)


In [24]:
"""
The two below queries are for displaying the output data in different ways.
One returns multiple rows when given a book thats present in several stores,
and the other only a single row. Both get the job done, and its really
only a matter of personal preference as to which is preferred.

The queries are represented in a parameterized form in order to prevent SQL injection
by adding clear distinctions between the query and user input.

"""


query = text(f"""                       
    SELECT
        Titel,
        Språk,
	    Pris AS 'Pris (kr)',
        [Vikt (g)],
        [Antal sidor],
        bö.ISBN,
        Namn AS 'Butik',
        Antal AS 'Antal i lager',
        Adress
        
    FROM
        -- join to get needed data
        Böcker bö
        INNER JOIN LagerSaldo la ON bö.ISBN = la.ISBN
        INNER JOIN Butiker bu ON la.ButikID = bu.ID
    WHERE
        Titel LIKE '%' + :search_words + '%'
""")


query_single_row = text(f"""                       
    SELECT
        Titel,
        Språk,
        Pris AS 'Pris (kr)',
        [Vikt (g)],
        [Antal sidor],
        bö.ISBN,
        STRING_AGG(bu.Namn, ' | ') AS 'Butik',
        STRING_AGG(Antal, ' | ') AS 'Antal i lager',
        STRING_AGG(Adress, ' | ') AS 'Adress'
    FROM
        -- join to get needed data
        Böcker bö
        INNER JOIN LagerSaldo la ON bö.ISBN = la.ISBN
        INNER JOIN Butiker bu ON la.ButikID = bu.ID
    -- group by for books that are present in multiple stores
    GROUP BY
        bö.ISBN,
        Titel,
        Språk,
        Pris,
        [Vikt (g)],
        [Antal sidor]
    HAVING
        Titel LIKE '%' + :search_words + '%'
""")

In [26]:
try:
    user_input = input('Search: ')

    # run bindparams for compatibility with the pandas implementation of parameterization
    parameterized_query = query_single_row.bindparams(search_words=user_input)
    
    # Connect to db, run query and return output as a dataframe 
    df = pd.read_sql_query(parameterized_query, con=engine.connect(), params={"search_words": f"%{user_input}%"})
        
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)

df

,Titel,Språk,Pris (kr),Vikt (g),Antal sidor,ISBN,Butik,Antal i lager,Adress
0,Liftoff,Engelska,331,646,304,9780062979971,Booked Up | Booktopia,2 | 6,"Kungsportsavenyn 23, 411 36 Göteborg | Söderma..."
